In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
sys.path.append("../")

In [ ]:
import os
from os.path import join, basename
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Any
from matplotlib import cm

from utils import load_syn_images, show_single_image, show_multiple_images
from photometric.estimate_alb_nrm import estimate_alb_nrm
from photometric.check_integrability import check_integrability
from photometric.photometric_stereo import photometric_stereo

### Monkey gray

In [ ]:
albedo, normals, height_map, SE = photometric_stereo(
    '../MonkeyGray/', shadow_trick=True, show=False, return_cache=True
)

In [ ]:
albedo.shape, normals.shape

In [ ]:
show_single_image(
    albedo, grayscale=True, xticks=False, yticks=False,
    save=True, path="../results/MonkeyGray_albedo.png", figsize=(5, 5)
)

In [ ]:
show_multiple_images(
    [normals[:, :, 0], normals[:, :, 1], normals[:, :, 2]], grid=(1, 3), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/MonkeyGray_normals.png", figsize=(12, 4)
)

#### Expt with varying $n$

In [ ]:
all_images = os.listdir('../MonkeyGray/')

In [ ]:
num_images = [3, 5, 10, 20, 40, 80, 100, len(all_images)]

In [ ]:
num_images

In [ ]:
threshold = 0.005

In [ ]:
albedos = []
outliers = []

for n in num_images:
    files = all_images[:n]

    albedo, normals, height_map, SE = photometric_stereo(
        image_dir="../MonkeyGray/", files=files, shadow_trick=True, show=True, return_cache=True
    )
    albedos.append(albedo)
    outliers.append(np.sum(SE > threshold))

In [ ]:
len(albedos), albedos[0].shape

In [ ]:
(albedos[0] == albedos[-1]).all()

In [ ]:
show_multiple_images(
    albedos, grid=(1, len(num_images)), grayscale=True, figsize=(20, 4),
    xticks=False, yticks=False, save=False, path="../results/MonkeyGray_n_normals.png",
)

In [ ]:
fig, ax = plt.subplots(1, len(num_images), figsize=(20, 7))

for i, x in enumerate(albedos):
    ax[i].imshow(x, cmap="gray")
    ax[i].set_title(f"$n = {num_images[i]}$")
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.savefig("../results/MonkeyGray_n_albedos.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.set_title("Number of outliers with varying $n$ for MonkeyGray")
ax.set_xlabel("Number of images $n$")
ax.set_ylabel("Number of outliers (SE$(x, y)$ > threshold = 0.005)")

ax.plot(num_images, outliers, 'o--')
ax.set_xticks(num_images)
ax.grid()

plt.savefig("../results/MonkeyGray_outliers.png", bbox_inches="tight")
plt.show()